In [1]:
#GC -- 06/24
#Calcualtes climate deltas (i.e., changes from the 2010-2020) period
#for pr, temp and wet days. Run on office desktop so some file paths may need 
# checking. Note the CC data is quite large (~1TB) and lives on an ext. HDD 
# connected to the office desktop, hence the platform specific setup. 

import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import os
import rioxarray as rxr
import rasterio as rio

In [71]:
#Create baseline .nc files for 2010-2020 time period. Will need to swap some 
# file paths for tmax, pr

#Set relevant directory
os.chdir("D:\\CanDCSensemble\\historical\\wet_days\\")
#Read in historic data from CMIP6 which goes to 2014.
nc_ls=["2010_tot.nc","2011_tot.nc","2012_tot.nc","2013_tot.nc","2014_tot.nc"]

#hist_tmax is a bad variable name as sometimes the var is wet days or
#precip. Oops
for nc_fp in nc_ls:
    hist_tmax=xr.open_dataset(nc_fp)
    if nc_fp=="2010_tot.nc":
        tot_nc=hist_tmax
    else:
        hist_tmax["time"]=tot_nc["time"]
        tot_nc+=hist_tmax
tot_hist_tmax=tot_nc

#For 2015-2020 we average all the SSP scenarios to get the baseline.
#Here we're just iterating through to do the averaging for each year
nc_ls=["2015_tot.nc","2016_tot.nc","2017_tot.nc","2018_tot.nc","2019_tot.nc","2020_tot.nc"]
for nc_fp in nc_ls:
    for wd in ["D:\\CanDCSensemble\\ssp126\\wet_days\\","D:\\CanDCSensemble\\ssp245\\wet_days\\","D:\\CanDCSensemble\\ssp585\\wet_days\\"]:
        os.chdir(wd)
        yearly_tmax=xr.open_dataset(nc_fp)
        if wd=="D:\\CanDCSensemble\\ssp126\\wet_days\\":
            tot_nc=yearly_tmax
        else:
            yearly_tmax["time"]=tot_nc["time"]
            tot_nc+=yearly_tmax
    yearly_nc=tot_nc/3
    if nc_fp=="2015_tot.nc":
        tot_ssp_tmax=yearly_nc
    else:
        yearly_nc["time"]=tot_ssp_tmax["time"]
        tot_ssp_tmax+=yearly_nc

#Average all 11 years to get the historic baseline
tot_ssp_tmax['pr']=tot_ssp_tmax['pr']/11

#Wrangle time to var and save
tot_ssp_tmax['time']=np.arange(1,366)
tot_ssp_tmax.to_netcdf("D:\\CanDCSensemble\\historical\\wet_days_avg_bline.nc",engine="scipy")


In [77]:
#Create .nc files for 21 year time frames. Change sy and ey and swap file paths 
#for var ssp and other vars

sy=2020
ey=2040

#Iterate through years in a 21 year time period and average 
os.chdir("D:\\CanDCSensemble\\ssp126\\wet_days\\")
for year in np.arange(sy,ey+1):
    temp_tmax=xr.open_dataset(str(year)+"_tot.nc")
    if year==sy:
        tot_tmax=temp_tmax
    else: 
        temp_tmax["time"]=tot_tmax["time"]
        tot_tmax+=temp_tmax
    
tot_tmax=tot_tmax/21

#Wrangle time and save
tot_tmax['time']=np.arange(1,366)
tot_tmax.to_netcdf("D:\\CanDCSensemble\\ssp126\\wet_days_avg_"+str(sy)+"_"+str(ey)+".nc",engine='scipy')

del tot_tmax

    

'\nos.chdir("D:\\CanDCSensemble\\ssp585\\wet_days\\")\nfor year in np.arange(sy,ey+1):\n    temp_tmax=xr.open_dataset(str(year)+"_tot.nc")\n    if year==sy:\n        tot_tmax=temp_tmax\n    else: \n        temp_tmax["time"]=tot_tmax["time"]\n        tot_tmax+=temp_tmax\n    \ntot_tmax=tot_tmax/21\n\ntot_tmax[\'time\']=np.arange(1,366)\ntot_tmax.to_netcdf("D:\\CanDCSensemble\\ssp585\\_day_avg_"+str(sy)+"_"+str(ey)+".nc")\n'

In [23]:
#Read in this file as we want to match the projection/grid of ERA5 which is what 
#the baseline wildfire model run uses. 
E5_test=xr.open_dataset("C:\\Users\\GiovanniCorti\\Documents\\BC_T2m2020.nc")
E5_test.rio.write_crs("epsg:4326", inplace=True)
#Middle day of each month
doy_15=[15,46,74,105,135,166,196,227,258,288,319,349]

#Iterate thru SSPs and vars, there are some issue with doing all at once
#due to ram limits.

#Will need to tweak vars for wet days but the logic is the same as for
#precip and the output will be a ratio. 
for ssp in ['ssp126','ssp245','ssp585']:
    for var in ['pr']:#['tasmax','pr']:
        if var=='tasmax':
            ab_var='tmax'
        else:
            ab_var=var
        for years in [[2020,2040],[2040,2060],[2060,2080],[2080,2100]]:
            bline=xr.open_dataset("D:\\CanDCSensemble\\historical\\"+ab_var+"_day_avg_bline.nc")
            tf_data=xr.open_dataset("D:\\CanDCSensemble\\"+ssp+"\\"+ab_var+"_day_avg_"+str(years[0])+"_"+str(years[1])+".nc")
            #Calc delta, done differently for tmax and precip
            
            #Smooth with 91 day window. Here I pad the deltas such that time dimension 
            #is wrapped. In otherwords Jan 1 avg will use some nov/dec days from the
            #end of the time dimension
            if var=='tasmax':
                #temp is calcualted as a delta
                delta=tf_data-bline
                delta=delta.pad(time=45,mode='wrap').tasmax.rolling(time=91,center=True).mean()[45:45+365,:,:]
            elif var=='pr':
                #Pr is calculated as a ratio
                delta=tf_data/bline
                delta=delta.pad(time=45,mode='wrap').pr.rolling(time=91,center=True).mean()[45:45+365,:,:]
                
            #Write CRS and reproject to ERA5 geom
            delta.rio.write_crs("epsg:4326", inplace=True)
            delta=delta.rio.reproject_match(E5_test,resampling=rio.enums.Resampling.average)
            
            #Mask out no data values
            delta=delta.where(delta<10000)
            
            #Select 15th day of each month
            m_delta=delta.isel(time=doy_15)
            #Wrangle time for each month
            m_delta['time']=np.arange(1,13)
        
            #Metadata cleanup
            m_delta.rio.write_nodata(None,inplace=True)
            m_delta=m_delta.to_dataset()
            m_delta.rio.write_crs("epsg:4326", inplace=True)
            
            #Save netcdf files
            m_delta.to_netcdf("D:\\CanDCSensemble\\Deltas\\"+ssp+"_"+ab_var+"_"+str(years[0])+"_"+str(years[1])+".nc")
            